In [ ]:
# preamble to be able to run notebooks in Jupyter and Colab
try:
    from google.colab import drive
    import sys
    
    drive.mount('/content/drive')
    notes_home = "/content/drive/Shared drives/CSC310/ds/notes/"
    user_home = "/content/drive/My Drive/"
    
    sys.path.insert(1,notes_home) # let the notebook access the notes folder

except ModuleNotFoundError:
    notes_home = "" # running native Jupyter environment -- notes home is the same as the notebook
    user_home = ""  # under Jupyter we assume the user directory is the same as the notebook

# NLP & ML

We saw that we convert text document into a ‘vector model’ (bag-of-words).

The vector model allows us to perform mathematical analysis on documents - “which documents are similar to each other?”

> Next question: can we construct machine learning models on document collections using the vector model?

**Yes!** We can construct classifiers.


Consider again our news article data set.

We would like to construct a classifier that can correctly classifier political and science documents.

We will begin with our KNN algorithm (k nearest neighbors). Since documents are considered point in an n-dimensional space KNN seems well suited for this problem.

## KNN

In [ ]:
# setup
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from assets.confint import classification_confint
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import PorterStemmer
from sklearn.datasets import fetch_20newsgroups

In [ ]:
print("******** data **********")

# get the newsgroup database
cats = ['talk.politics.misc', 'sci.space']
newsgroups_train = fetch_20newsgroups(subset='train', 
                                      remove=('headers', 'footers', 'quotes'),
                                      categories=cats)

# extract into dataframes
texts = pd.DataFrame(newsgroups_train.data, columns=['text'])
labels = pd.DataFrame(newsgroups_train.target, columns=['label'])['label'].apply(lambda x: cats[x])
texts.head()

******** data **********


/Users/lutz/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.datasets.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.datasets. Anything that cannot be imported from sklearn.datasets is now part of the private API.
  warnings.warn(message, FutureWarning)


,text
0,\nIn billions of dollars (%GNP):\nyear GNP ...
1,ajteel@dendrite.cs.Colorado.EDU (A.J. Teel) w...
2,\nMy opinion is this: In a society whose econ...
3,"Ahhh, remember the days of Yesterday? When we..."
4,"\n""...a la Chrysler""?? Okay kids, to the near..."


In [ ]:
print("******** docarray **********")

# build the stemmer object
stemmer = PorterStemmer()
# get the default text analyzer from CountVectorizer
analyzer = vectorizer = CountVectorizer(analyzer = "word", token_pattern = "[a-zA-Z]+").build_analyzer()

# build a new analyzer that stems using the default analyzer to create the words to be stemmed
def stemmed_words(doc):
    return (stemmer.stem(w) for w in analyzer(doc))

# build docarrayu
vectorizer = CountVectorizer(analyzer=stemmed_words,
                                 binary=True,
                                 min_df=2)
docarray = vectorizer.fit_transform(texts.loc[:,'text']).toarray()
docarray.shape

******** docarray **********


(1058, 6267)

In [ ]:
print("******** model **********")


# KNN
model = KNeighborsClassifier()

# grid search
param_grid = {'n_neighbors': list(range(1,15,3))}
grid = GridSearchCV(model, param_grid, cv=2, verbose=10, n_jobs=-1)
grid.fit(docarray, labels)
print("Grid Search: best parameters: {}".format(grid.best_params_))

******** model **********
Fitting 2 folds for each of 5 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:   15.5s remaining:   15.5s
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:   15.5s remaining:    6.7s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   19.8s finished


Grid Search: best parameters: {'n_neighbors': 7}


In [ ]:
print("******** Accuracy **********")

# accuracy of best model with confidence interval
best_model = grid.best_estimator_
predict_y = best_model.predict(docarray)
acc = accuracy_score(labels, predict_y)
lb,ub = classification_confint(acc,docarray.shape[0])
print("Accuracy: {:3.2f} ({:3.2f},{:3.2f})".format(acc,lb,ub))

******** Accuracy **********
Accuracy: 0.86 (0.84,0.88)


In [ ]:
print("******** confusion matrix **********")

# build the confusion matrix
cm = confusion_matrix(labels, predict_y, labels=cats)
cm_df = pd.DataFrame(cm, index=cats, columns=cats)
print("Confusion Matrix:\n{}".format(cm_df))

******** confusion matrix **********
Confusion Matrix:
                    talk.politics.misc  sci.space
talk.politics.misc                 514         79
sci.space                           73        392


## Naive Bayes (NB)

* “Standard” model for text processing
* Fast to train, has no problems with very high dimensional data
* NB is a classification technique based on Bayes’ Theorem with an assumption of independence among predictors. 
* In simple terms, a NB classifier assumes that the presence of a particular feature in a class is unrelated to the presence of any other feature. 
* For example, a fruit may be considered to be an apple if it is red, round, and about 3 inches in diameter. Even if these features depend on each other or upon the existence of the other features, all of these properties independently contribute to the probability that this fruit is an apple and that is why it is known as ‘Naive’.


### The Mathematics

[Source](https://www.analyticsvidhya.com/blog/2017/09/naive-bayes-explained)

* Bayes theorem provides a way of calculating posterior probability $P(c|x)$ from $P(c)$, $P(x)$ and $P(x|c)$. Look at the equation below, where
  * $P(c|x)$ is the posterior probability of class (c, target) given predictor (x, attributes).
  * $P(c)$ is the prior probability of class.
  * $P(x|c)$ is the likelihood which is the probability of predictor given class.
  * $P(x)$ is the prior probability of predictor.

<img src="https://www.analyticsvidhya.com/wp-content/uploads/2015/09/Bayes_rule-300x172.png" width="400" height="400">

### Example

Let's assume we have a predictor `Weather` and a target `Play` that contains classes (left table below).  

<img src="https://www.analyticsvidhya.com/wp-content/uploads/2015/08/Bayes_41.png">

We want to compute if we play tennis when sunny.  That is we compute the two probabilities,
1. $P(Yes|Sunny)$
1. $P(No|Sunny)$
and then pick the statement with the higher probability.

Basically, NB just counts, let's look at $P(Yes|Sunny)$,

$P(Yes|Sunny) = \frac{P(Sunny|Yes)P(Yes)}{P(Sunny)} = \frac{3/9\times 9/14}{5/14} = \frac{.33 \times .64}{.36}=.60$

Now, let's look at $P(No|Sunny)$,

$P(No|Sunny) = \frac{P(Sunny|No)P(No)}{P(Sunny)} = \frac{2/5\times 5/14}{5/14} = \frac{.40 \times .36}{.36}=.40$

We are playing tennis when sunny because the posterior probability $P(Yes|Sunny)$ is higher.

Let’s take our text classification problem and use a Naive Bayes classifier on it.

The setup and data prep is the same as in the case of the KNN classifier.

In [ ]:
## Naive Bayes

# setup
import pandas as pd
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from assets.confint import classification_confint
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import PorterStemmer
from sklearn.datasets import fetch_20newsgroups

print("******** data **********")

# get the newsgroup database
cats = ['talk.politics.misc', 'sci.space']
newsgroups_train = fetch_20newsgroups(subset='train', 
                                      remove=('headers', 'footers', 'quotes'),
                                      categories=cats)

# extract into dataframes
texts = pd.DataFrame(newsgroups_train.data, columns=['text'])
labels = pd.DataFrame(newsgroups_train.target, columns=['label'])['label'].apply(lambda x: cats[x])

print("******** docarray **********")

# build the stemmer object
stemmer = PorterStemmer()
# get the default text analyzer from CountVectorizer
analyzer = vectorizer = CountVectorizer(analyzer = "word", token_pattern = "[a-zA-Z]+").build_analyzer()

# build a new analyzer that stems using the default analyzer to create the words to be stemmed
def stemmed_words(doc):
    return (stemmer.stem(w) for w in analyzer(doc))

# build docarrayu
vectorizer = CountVectorizer(analyzer=stemmed_words,
                                 binary=True,
                                 min_df=2)
docarray = vectorizer.fit_transform(texts.loc[:,'text']).toarray()
docarray.shape

print("******** model **********")


# Naive Bayes
model = MultinomialNB()
# NOTE: NB does not have any hyper-parameters - no overfitting - no searching over parameter space!
model.fit(docarray, labels)


print("******** Accuracy **********")

# accuracy of best model with confidence interval
best_model = model
predict_y = best_model.predict(docarray)
acc = accuracy_score(labels, predict_y)
lb,ub = classification_confint(acc,docarray.shape[0])
print("Accuracy: {:3.2f} ({:3.2f},{:3.2f})".format(acc,lb,ub))

print("******** confusion matrix **********")

# build the confusion matrix
cm = confusion_matrix(labels, predict_y, labels=cats)
cm_df = pd.DataFrame(cm, index=cats, columns=cats)
print("Confusion Matrix:\n{}".format(cm_df))

******** data **********
******** docarray **********
******** model **********
******** Accuracy **********
Accuracy: 0.95 (0.94,0.97)
******** confusion matrix **********
Confusion Matrix:
                    talk.politics.misc  sci.space
talk.politics.misc                 562         31
sci.space                           19        446


Trains very fast and has a higher accuracy than KNN and the difference in accuracy is statistically significant!

> NB does not have any hyper-parameters - no overfitting - no searching over parameter space!

Hint: Try cross-validating the NB model - you will find that the fold accuracies and the mean accuracy will fall into the CI computed above.



# Team Exercise

For this exercise you will build a classifier that can distinguish real news from fake news. A training set for this is available here:
https://raw.githubusercontent.com/lutzhamel/fake-news/master/data/fake_or_real_news.csv

The fields you are interested in are ‘text’ and ‘label’ with the obvious interpretations. 

Here are the action items for this exercise:
* Use the vector model and text preprocessing techniques from class to construct a training data set.
* Determine the dimensions of your vector model and print out the first 10 dimensions
* Use that training data set to construct a Naive Bayes classifier.  
* Compute the accuracy and 95% CI for the classifier.
* Try your analysis with and without data preprocessing, is there a difference in accuracy of the models.

The data set contains a large number of articles (takes a long time to train), you can downsample this to something like a 1,000 articles or so in order to speed up training and evaluation (hint: use shuffle).

You are free to pick your own team (max three members)

Extra Credit:  Try the same thing but instead of ‘text’ use ‘title’ for your training text.  How does a classifier built on this data set compare to the original classifier.
